In [1]:
import numpy as np
import pyCalc as p
import random
import timeit

In [2]:
def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

X = np.array([  [0,0,1],
                    [0,1,1],
                    [1,0,1],
                    [1,1,1] ])
y = np.array([[0,0,1,1]]).T
np.random.seed(1)
syn0 = 2*np.random.random((3,1)) - 1
def native():
    global syn0, y, X
    for iter in range(10000):
        l0 = X
        l1 = nonlin(np.dot(l0,syn0))
        l1_error = y - l1
        l1_delta = l1_error * nonlin(l1,True)
        syn0 += np.dot(l0.T,l1_delta)



In [3]:
p.setMode("quiet")
xs = p.var("constant",np.array([
                                   [1,0,0],
                                   [1,1,0],
                                   [1,0,1],
                                   [1,1,1]
                               ]
                               , dtype= float))
ys = p.var("constant", np.array(
                                [0,0,1,1]
                                , dtype=float))
half = p.var("constant", np.array(
                                [
                                   0.5
                               ]
                                , dtype=float))
np.random.seed(1)
weights =2*np.random.random((3,1)) - 1
ws = p.var("independent", 3,1)

l0 = p.function(xs,ws, "*") #4x1
l1 = p.function(l0, "sigmoid") #4x1
l1_error = p.function(ys.T(), l1, "-")#4x1
sq_err = p.function(l1_error, l1_error, "colwise*")#4x1
cost = p.function(p.function(sq_err,half,"*"),"colsum")


sig = p.function(l1, "sigmoid_r")
l1_delta = p.function(sig, l1_error, "colwise*")
tmp = p.function(xs.T(),l1_delta, "*")
weights = weights + tmp.value(dict([(ws,weights)]))
def pgcg_sym():
    global weights, tmp
    for i in range(10000):
        weights = weights + tmp.value()
        
def pgcg_num():
    global weights, cost,ws
    for i in range(10000):
        weights = weights + p.differeniate(cost,ws,weights)

Exception ignored in: 'pyCalc.cError'
ValueError: <invalid operation: colwise* >left is a 4x1 matrix but right is a 4x1 matrix
Exception ignored in: 'pyCalc.cError'
ValueError: <invalid operation: colwise* >left is a 4x1 matrix but right is a 4x1 matrix


In [4]:
timeit.timeit(native,number=1000)

133.20914459000005

In [5]:
timeit.timeit(pgcg_sym,number=1000)

573.0958234359996

weights =2*np.random.random((3,1)) - 1
timeit.timeit(pgcg_num,number=10)